# Search-Engine-from-Scratch

# NAME(s) and NIA(s) 

**Author :** Stephen Appiah.
         Umar Muhammad.
**NIA :** 206637.
      208244.
    

#### Load Python packages
Let's first import all the packages that you will need during this assignment.

In [12]:
# if you do not have 'nltk', the following command should work "python -m pip install nltk"
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\work\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\work\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [13]:
from collections import defaultdict
from array import array
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import math
import numpy as np
import collections
import time
from numpy import linalg as la

#### Load data into memory
Import the 500 Wikipedia articles (one article per line). For each article we have the document id, document title and document body separated by "|" character.

In [14]:
docs_path = 'inputs/documents-corpus.tsv'
with open(docs_path) as fp:
    lines = fp.readlines()
lines = [l.strip().replace(' +', ' ') for l in lines]

In [15]:
print("Total number of Wikipedia articles in the corpus: {}".format(len(lines)))

Total number of Wikipedia articles in the corpus: 500


# Building the Index


Processing the data

Define functions to clean the text (tokenization / bigram/ lowercase /remove punctuation /stemming)

In [16]:
stemmer = PorterStemmer()
stop_words = set(stopwords.words("english"))

def remove_stopwords(words):
    return [w for w in words if w.lower() not in stop_words]

def stem_each(words):
    return [stemmer.stem(w) for w in words]

def remove_white_space(text):
    return ' '.join(text.split())

In [17]:
def preprocess(text):
  """
  Preprocess the article text (title + body) removing stop words, stemming,
  transforming in lowercase and return the tokens of the text.

  Argument:
  line -- string (text) to be preprocessed

  Returns:
  line - a list of tokens corresponding to the input text after the preprocessing
  """
  stemmer = PorterStemmer()
  stop_words = set(stopwords.words("english"))
  text=  text.lower()
  text = text.replace('#', '')
  text=  text.split()
  text= remove_stopwords(text)
  text= stem_each(text)
  return text

In [18]:
#Eample
phrase = "Smart-assistants like Apple’s Siri and Amazon’s Alexa recognize patterns in speech thanks to voice recognition, then infer meaning and provide a useful response."
print(preprocess(phrase))

['smart-assist', 'like', 'apple’', 'siri', 'amazon’', 'alexa', 'recogn', 'pattern', 'speech', 'thank', 'voic', 'recognition,', 'infer', 'mean', 'provid', 'use', 'response.']


#### Inverted Index Construction

So the first step in building a text search engine is assembling an inverted index. An inverted index is a data structure that maps tokens to the documents they appear in. In this context, we can consider a token to simply be a word, so an inverted index, at its most basic, is just something that takes in a word, and returns to us a list of the documents it appears in.

In [48]:
def inverted_index(lines, num_documents):
    """
    Implement the inverted index and compute tf, df and idf
    
    Argument:
    lines -- collection of Wikipedia articles
    num_documents -- total number of documents
    
    Returns:
    index - the inverted index (implemented through a Python dictionary) containing terms as keys and the corresponding
    list of document these keys appears in (and the positions) as values.
    tf - normalized term frequency for each term in each document
    df - number of documents each term appear in
    idf - inverse document frequency of each term
    """
    index = defaultdict(list)
    tf = defaultdict(list)
    df = defaultdict(int)
    title_index = defaultdict(str)
    idf = defaultdict(float)
    
    for line in lines:
        line_arr = line.split("|")
        page_id = int(line_arr[0])
        terms = preprocess(''.join(line_arr[1:])) 
        title = line_arr[1]
        title_index[page_id]=title
        
        ## As suggested in class, we are using a current_page_index ==> { ‘term1’: [current_doc, [list of positions]], ...,‘term_n’: [current_doc, [list of positions]]}
        ## current_page_index ==> { ‘web’: [1, [0]], ‘retrieval’: [1, [1,4]], ‘information’: [1, [2]]}
        current_page_index = {}
        for position, term in enumerate(terms):
            try:
                current_page_index[term][page_id].append(position)
            except:
                current_page_index[term]=[page_id, array('I',[position])] #'I' indicates unsigned int (int in Python)
            
        # normalized the term frequencies
        norm = 0
        for term, posting in current_page_index.items():
            norm += len(posting[1]) ** 2
        norm = math.sqrt(norm)
        
        #calculate the tf(dividing the term frequency by the above computed norm) and df weights
        for term, posting in current_page_index.items():
            tf[term].append(np.round(len(posting[1])/norm,4))
            df[term] = df[term] + 1 
            
        #merge the current page index with the main index
        for term_page, posting_page in current_page_index.items():
            index[term_page].append(posting_page)
        
        # Compute IDF
        for term in df:
            idf[term] = np.round(np.log(float(num_documents/df[term])), 4)
            
    return index, tf, df, idf, title_index

In [49]:
num_documents = len(lines)
index, tf, df, idf, title_index = inverted_index(lines, num_documents)

In [41]:
#Eample
print("First 5 Index results for the term 'research': \n{}".format(index['research'][:5]))

First 5 Index results for the term 'research': 
[[33, array('I', [76])], [104, array('I', [633])], [131, array('I', [1257])], [139, array('I', [84])], [183, array('I', [1411])]]


# Querying the Index


#### Ranking

When searching in a search engine, we are interested in obtain the results sorted by relevance or by some other criteria.

In [57]:
def rank_documents(terms, docs, index, idf, tf, title_index):
    """
    Perform the ranking of the results of a search based on the tf-idf weights
    
    Argument:
    terms -- list of query terms
    docs -- list of documents, to rank, matching the query
    index -- inverted index data structure
    idf -- inverted document frequencies
    tf -- term frequencies
    title_index -- mapping between page id and page title
    
    Returns:
    Print the list of ranked documents
    """
    
    doc_vectors = defaultdict(lambda: [0] * len(terms))
    query_vector = [0] * len(terms)
    query_terms_count = collections.Counter(terms)
    query_norm = la.norm(list(query_terms_count.values()))
    for termIndex, term in enumerate(terms):
        if term not in index: continue
        query_vector[termIndex]= query_terms_count[term]/query_norm * idf[term]
        for doc_index, (doc, postings) in enumerate(index[term]):
            if doc in docs:
                doc_vectors[doc][termIndex] = tf[term][doc_index] * idf[term]  # TODO: check if multiply for idf
    
    doc_scores=[[np.dot(curDocVec, query_vector), doc] for doc, curDocVec in doc_vectors.items() ]
    doc_scores.sort(reverse=True)
    result_docs = [x[1] for x in doc_scores]
    if len(result_docs) == 0: 
        print("No results found, try again")
        query = input()
        docs = search(query, index)
    return result_docs

#### Search

Enter queries to retrieved the top 10 relevant links based on the cosine similarity score (ltc.ltn schema)

In [61]:
def search(query, index):
    """
    output is the list of documents that contain any of the query terms. 
    So, we will get the list of documents for each query term, and take the union of them.
    """
    query = preprocess(query)
    docs = set()
    for term in query:
        try:                      
            term_docs=[posting[0] for posting in index[term]]
            docs |= set(term_docs)
        except:
            pass
    docs = list(docs)
    ranked_docs = rank_documents(query, docs, index, idf, tf, title_index)
    return ranked_docs

In [62]:
print("Insert your query (i.e.: Computer Science):\n")
query = input()
docs = search(query, index)
top = 10

print("\n======================\nSample of {} results out of {} for the searched query:\n".format(top, len(docs)))
for d_id in docs[:top]:
    print("page_id= {} - page_title: {}".format(d_id, title_index[d_id]))

Insert your query (i.e.: Computer Science):

Computer Science

Sample of 10 results out of 345 for the searched query:

page_id= 2583 - page_title: Association of Synchronous Data Formats
page_id= 297 - page_title: ACM Portal
page_id= 2086 - page_title: Aperiodic finite state automaton
page_id= 2784 - page_title: Australian Partnership for Advanced Computing
page_id= 1220 - page_title: Aggregate function
page_id= 1702 - page_title: American flag sort
page_id= 647 - page_title: A Sharp   NET 
page_id= 956 - page_title: Adaptive Behavior
page_id= 3939 - page_title: Bipolar violation
page_id= 3190 - page_title: BTSharp
